# Import Libraries

In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

sns.set()

# Data Exploration

In [ ]:
train_male = len(os.listdir("/kaggle/input/gender-classification-dataset/Training/male"))
train_female = len(os.listdir("/kaggle/input/gender-classification-dataset/Training/female"))
valid_male = len(os.listdir("/kaggle/input/gender-classification-dataset/Validation/male"))
valid_female = len(os.listdir("/kaggle/input/gender-classification-dataset/Validation/female"))
print(f"There are {train_male} male and {train_female} female in the training set")
print(f"There are {valid_male} male and {valid_female} female in the validation set")

In [ ]:
shapes = np.array([cv2.imread(path, -1).shape for path in np.random.choice(glob.glob("/kaggle/input/gender-classification-dataset/**/**/*"), 1000, replace=False)])
if not np.all(shapes==shapes[0]):
    print("Not all images have the same shape")
else:
    print(f"Almost all images are of shape {shapes[0]}")

In [ ]:
np.min(shapes, axis=0), np.max(shapes, axis=0), np.mean(shapes, axis=0), np.median(shapes, axis=0)

In [ ]:
median_shape = np.median(shapes, axis=0).astype(int)
min_shape = np.min(shapes, axis=0).astype(int)

In [ ]:
random_imgs = np.random.choice(glob.glob("/kaggle/input/gender-classification-dataset/**/**/*"), 30, replace=False)
plt.figure(figsize = (18, 5))
plt.suptitle(f"Example of {len(random_imgs)} random images from all the data")
for i in range(len(random_imgs)):
    plt.subplot(3,10,i+1)
    img = cv2.imread(random_imgs[i], -1)[...,::-1]/255.
    noise = np.random.normal(0, 0.2, size=img.shape)
    img = img + noise
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')

plt.show()

# Data Loader

In [ ]:
female_path = glob.glob("/kaggle/input/gender-classification-dataset/Training/female/*")
female_label = [1] * len(female_path)
male_path = glob.glob("/kaggle/input/gender-classification-dataset/Training/male/*")
male_label = [0] * len(male_path)
all_pathes = female_path + male_path
all_labels = female_label + male_label

We will create a class to load and generate data to our model. The class will be responsible of reading, shuffling and feeding the data into the fully connected network. 

In [ ]:
class DataLoader:
    def __init__(self, data_path, batch_size, shape):
        self._X = self._read_data_path(data_path)
        self._batch_size = batch_size
        self.nb_iterations = self.__len__()//batch_size
        self._shape = shape
    
    def __len__(self):
        return len(self._X)
    
    def get_shape(self):
        return self._shape + (3, )
    
    def _read_data_path(self, data_path):
        """
        This method takes the path to training or validation data,
        and return two arrays X and Y. X contains the full path to
        all images, and Y contains the corresponding labels:
        0 for male, 1 for female.
        
        data_path: str, the path to the training or validation dataset
        return:
        tuple (X, Y) of type np.array each
        """
        female_path = glob.glob(f"{data_path}/female/*")
        male_path = glob.glob(f"{data_path}/male/*")
        all_pathes = female_path + male_path
        return all_pathes
    

    def _read_single_image(self, img_path):
        """
        This method takes an image path and a label, read the image and retrun it with the label.
        The image should be converted into gray scale, resized into self._shape,
        normalized to 1, and vectorized.
        """
        img = tf.io.decode_png(
            tf.io.read_file(img_path), channels=3, dtype=tf.uint8
        )
        resized_img = tf.image.resize(img, self._shape) / 255
        noise = tf.random.normal(shape=self._shape + (3,), mean=0.0, stddev=0.2)
        
        return tf.clip_by_value(resized_img + noise, 0, 1), resized_img
    
    def get_dataset(self):
        """
        This method should create a dataset from the image path and label,
        shuffle them, repeat the dataset, read the actual images,
        create batches and return a dataset to be fed to the model. 
        """
        dataset = tf.data.Dataset.from_tensor_slices(self._X)
        dataset = dataset.shuffle(buffer_size=self.__len__(), reshuffle_each_iteration=True).repeat()
        dataset = dataset.map(self._read_single_image, tf.data.AUTOTUNE)
        dataset = dataset.batch(self._batch_size, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset

# Network

In [ ]:
def conv_bn_relu(x, feature_maps, strides, kernel_size):
    x = tf.keras.layers.Conv2D(
          filters=feature_maps, kernel_size=kernel_size,
          strides=strides, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x


def encoder_residual_block(x, feature_maps_list, kernel_size_list, drop_rate, downsample=True):
    x = conv_bn_relu(x, feature_maps_list[-1], strides=2 if downsample else 1, kernel_size=1)
    residual = x
    
    if drop_rate > 0:
        x = tf.keras.layers.SpatialDropout2D(drop_rate)(x)

    for i in range(len(feature_maps_list)):
        x = conv_bn_relu(x, feature_maps_list[i], strides=1, kernel_size=kernel_size_list[i])

    return tf.keras.layers.Add()([x, residual])


def decoder_residual_block(x, encoder_block, feature_maps_list, kernel_size_list, drop_rate):
    x = tf.keras.layers.UpSampling2D()(x)
    if x.shape[1:-1] != encoder_block.shape[1:-1]:
        x = tf.image.resize_with_crop_or_pad(x, encoder_block.shape[1], encoder_block.shape[2])
    
    residual = x
    
    if x.shape[-1] != feature_maps_list[-1]:
        residual = conv_bn_relu(x, feature_maps_list[-1], strides=1, kernel_size=1)

    if drop_rate > 0:
        x = tf.keras.layers.SpatialDropout2D(drop_rate)(x)

    for i in range(len(feature_maps_list)):
        x = conv_bn_relu(x, feature_maps_list[i], strides=1, kernel_size=kernel_size_list[i])

    return tf.keras.layers.Add()([x, residual])


In [ ]:
def create_encoder_decoder_model(input_shape, config):
    input_image = tf.keras.layers.Input(input_shape)

    encoded = input_image
    encoder_layers = []
    for i in range(len(config["encoder_fmaps"])):
        encoded = encoder_residual_block(encoded, config["encoder_fmaps"][i], config["encoder_kernels"][i], config["drop_rate"], downsample=(i!=0))
        encoder_layers.append(encoded)


    code = conv_bn_relu(encoded, config["code_fmaps"], config["code_stride"], config["code_kernel"])

    decoded = code
    decoder_layers = []
    for i in range(len(config["decoder_fmaps"])):
        decoded = decoder_residual_block(decoded, encoder_layers[-i-1], config["decoder_fmaps"][i], config["decoder_kernels"][i], config["drop_rate"])
        decoder_layers.append(decoded)

    output = tf.keras.layers.Conv2D(filters=3, kernel_size=3, strides=1, padding='same')(decoded)

    model = tf.keras.Model(input_image, output)
    return model

# Optimizer

In [ ]:
def create_optimizer(lr, nb_iterations, use_cosine_decay=False):
    """
    This function should create an Adam optimizer.
    If use_cosine_decay is True, it should apply a cosine_decay_with restart
    (look at tf.keras.optimizers.schedules.CosineDecayRestarts)
    with a cycle of 10 epochs, an alpha=0.01 and t_mul=2.0.
    This function should return an optimizer instance.
    """
    if use_cosine_decay:
        lr = tf.keras.optimizers.schedules.CosineDecayRestarts( lr, 10*nb_iterations, t_mul=1.0, m_mul=1.0, alpha=0.01,)
    optimizer = tf.keras.optimizers.Adam(lr)
    return optimizer

In [ ]:
batch_size=32
resize_shape=(120, 120)
training_path="/kaggle/input/gender-classification-dataset/Training"
validation_path="/kaggle/input/gender-classification-dataset/Validation"

training_data = DataLoader(training_path, batch_size, resize_shape)
validation_data = DataLoader(validation_path, batch_size, resize_shape)

In [ ]:
config={
    "encoder_fmaps": [[8, 8], [16, 16], [32, 32], [64, 64], [128, 128]],
    "encoder_kernels": [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]],

    "decoder_fmaps": [[128, 128], [64, 64], [32, 32], [16, 16], [8, 8]],
    "decoder_kernels": [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]],
    
    "code_fmaps": 256,
    "code_stride": 2,
    "code_kernel": 5,

    "drop_rate": 0}

model = create_encoder_decoder_model(training_data.get_shape(), config)
model.summary()

In [ ]:
learning_rate=0.01
optimizer = create_optimizer(learning_rate, training_data.nb_iterations, True)

In [ ]:
def psnr(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

def ssim(y_true, y_pred):
    return tf.image.ssim(y_true, y_pred, max_val=1.0)

In [ ]:
epochs=50

model.compile(optimizer=optimizer, loss="mse", metrics=['mae', psnr, ssim])

history = model.fit(training_data.get_dataset(),
                    validation_data=validation_data.get_dataset(),
                    epochs=epochs, steps_per_epoch=training_data.nb_iterations, validation_steps=validation_data.nb_iterations)

In [ ]:
train_loss = history.history['loss']
train_acc = history.history['accuracy']
valid_loss = history.history['val_loss']
valid_acc = history.history['val_accuracy']

In [ ]:
fig = plt.figure(figsize=(12, 6))
plt.suptitle("Learning Curves")
    
plt.subplot(121)
plt.title("cross entropy")
plt.plot(np.arange(1, len(train_loss)+1), train_loss, label='training', c='b')
plt.plot(np.arange(1, len(valid_loss)+1), valid_loss, label='validation', c='r')
plt.xlim(1, epochs + epochs//10)
plt.xticks(np.arange(0, epochs + epochs//10, epochs//10))
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper right')

plt.subplot(122)
plt.title("accuracy")
plt.plot(np.arange(1, len(train_acc)+1), train_acc, label='training', c='b')
plt.plot(np.arange(1, len(valid_acc)+1), valid_acc, label='validation', c='r')
plt.xlim(1, epochs + epochs//10)
plt.xticks(np.arange(0, epochs + epochs//10, epochs//10))
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='lower right')

# Test the model on new images

In [ ]:
fake_faces = []
for img_path in glob.glob("/kaggle/input/fakefaces/*"):
    img = cv2.cvtColor(cv2.imread(img_path, -1), cv2.COLOR_BGR2RGB)/255.
    noise = np.random.normal(0, 0.3, size=img.shape)
    img = img + noise
    img = np.clip(img, 0, 1)
    fake_faces.append(img)

In [ ]:
plt.figure(figsize=(24, 5))
for i in range(len(fake_faces)):
    plt.subplot(1, len(fake_faces), i+1)
    plt.imshow(fake_faces[i])
    plt.axis('off')

plt.show()

In [ ]:
batch_faces = []
for img in fake_faces:
    
    img = cv2.resize(img, resize_shape)
    batch_faces.append(img)
    
batch_faces = np.array(batch_faces)

In [ ]:
preds = model(batch_faces)
plt.figure(figsize=(24, 5))
for i in range(len(fake_faces)):
    plt.subplot(1, len(fake_faces), i+1)
    plt.imshow(preds[i])
    plt.axis('off')

plt.show()